In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
from sklearn.metrics import mean_squared_error,classification_report,f1_score,confusion_matrix,accuracy_score

import xgboost as xgb
import lightgbm as lgb

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate, learning_curve,train_test_split,GridSearchCV, cross_val_score, RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier

from sklearn.preprocessing import LabelEncoder

from warnings import filterwarnings
filterwarnings("ignore")

In [ ]:
df = pd.read_csv("../input/mice-protein-expression/Data_Cortex_Nuclear.csv")
df.head()

In [ ]:
df.drop(["MouseID"],axis=1,inplace=True)

In [ ]:
df.shape

In [ ]:
df.isna().sum()[50:]

In [ ]:
df.groupby(["class"]).mean()

In [ ]:
for i in range(len(df.columns[:-4])):
    if len(df[df[df.columns[i]].isna() == True].index)>0:
        for j in range(len(df[df[df.columns[i]].isna() == True].index)):
            index = df[df[df.columns[i]].isna() == True].index
            if "True" in str(df.iloc[index[0]:index[0]+1][df.columns[i]].isna()):
                df.iloc[index[0]:index[0]+1][df.columns[i]] = df.groupby(["class"]).mean()[df.columns[i]][list(df[index[0]:index[0]+1]["class"])[0]]

In [ ]:
df.isna().sum()[50:]

In [ ]:
le = LabelEncoder()
df["Genotype"] = le.fit_transform(df["Genotype"])
df["Treatment"] = le.fit_transform(df["Treatment"])
df["Behavior"] = le.fit_transform(df["Behavior"])
df["class"] = le.fit_transform(df["class"])

# Model

In [ ]:
log_model = LogisticRegression()

gbc_model = GradientBoostingClassifier(random_state=14)

rfc_model = RandomForestClassifier(criterion='gini', n_estimators=999,max_depth=4, random_state=14)

lgb_model = lgb.LGBMClassifier(num_iterations=550, learning_rate=0.01055,max_depth=3, random_state=14)

xgb_model = xgb.XGBClassifier(objective="binary:hinge")

gnb_model = GaussianNB()

mlpc_model = MLPClassifier(random_state=14)

svc_model = SVC(probability=True)

knn_model = KNeighborsClassifier(n_neighbors=19,leaf_size=20)

In [ ]:
models = [log_model, gbc_model, rfc_model, lgb_model, xgb_model, gnb_model, mlpc_model, svc_model,knn_model]
model_names = ["LogisticRegression","GradientBoostingClassifier","RandomForestClassifier","LGBMClassifier","XGBClassifier","GaussianNB","MLPClassifier","SVC","KNeighborsClassifier"]

x = df.drop('class', axis=1)
y = df['class']

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=42)

In [ ]:
print("TRAIN SCORE\n\n")
print("-----------------------------------------")
for i in range(len(models)):
    model = models[i].fit(x_train,y_train)
    print(model_names[i],"Score = %",model.score(x_train,y_train)*100)

In [ ]:
print("TEST SCORE\n\n")
print("-----------------------------------------")
for i in range(len(models)):
    model = models[i].fit(x_train,y_train)
    print(model_names[i],"Score = %",model.score(x_test,y_test)*100)